Teste feito com as seguintes observações
- Coluna de cabine foi utilizado uma abordagem de pegar a penas a letra da cabine
- Idades faltantes preenchidas com a média aritimética
- k = 30
- Acuracia: 58%

Importando as bibliotecas necessárias

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt

Lendo o arquivo de treino

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df

Definindo método para tratar das cabines

In [ ]:
def get_deck(cabin):
    if pd.isna(cabin):
        return None
    else:
        return cabin.split()[0][0]

Tratando as idades faltantes
 - Aplicado a media das idades nos registros faltantes

In [ ]:
df['Age'] = df['Age'].fillna(round(df['Age'].mean()))

In [ ]:
df

Definindo a coluna de cabine com base na função definida anteriormente

In [ ]:
df['Deck'] = df['Cabin'].apply(get_deck)

In [ ]:
df

Para as cabines desconhecidas, preencheremos com um X.\
E removemos a coluna 'Cabin'

In [ ]:
df['Deck'] = df['Deck'].fillna('X')
df = df.drop(columns=['Cabin'])

In [ ]:
df

Como no knn só trabalhamos com variaveis numéricas
- Transformaremos as colunas Deck, Name, Ticker e Embarked em variaveis dummies

In [ ]:
df = pd.get_dummies(df, columns=['Deck', 'Name', 'Ticket', 'Embarked'])

In [ ]:
df

Com a coluna de genero, iremos atribuir 0 para feminino e 1 para masculino 

In [ ]:
df['Sex'] = df['Sex'].replace({'female': 0, 'male': 1})

In [ ]:
df

Agora iremos definir o X e o y, removendo a coluna 'Survived' que é o alvo

In [ ]:
X = df.drop(['Survived',], axis=1)
y = df['Survived']

In [ ]:
X

In [ ]:
y

Separamos os dados para treino, 80% para treino e 20% para validação

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Cross Validation

In [ ]:
k_range = range(1, 31)
k_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=50, scoring='accuracy')
    k_scores.append(scores.mean())

melhor_k = k_range[k_scores.index(max(k_scores))]
melhor_score = max(k_scores)

print(f"Melhor valor de k: {melhor_k}")
print(f"Melhor acurácia: {melhor_score:.4f}")

plt.plot(k_range, k_scores)
plt.xlabel('Valor de K para KNN')
plt.ylabel('Acurácia Média com Validação Cruzada')
plt.title('Acurácia vs. Valor de K')
plt.show()

Definimos o modelo de classificação

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

Treinamos o modelo

In [ ]:
knn.fit(X_train, y_train)

Acuracia do modelo

In [ ]:
print(knn.score(X_test, y_test))

Predição do modelo

In [ ]:
y_pred = knn.predict(X_test)

Matriz de confusão

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Sobreviveu', 'Sobreviveu'], yticklabels=['Não Sobreviveu', 'Sobreviveu'])
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

Relatório de classificação

In [ ]:
print(classification_report(y_test, y_pred, target_names=['Não Sobreviveu', 'Sobreviveu']))